## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [ ]:
%load_ext autoreload
%autoreload 2

from ewxpwsdb.db.models import WeatherStation, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table
station_file = '../data/test_stations.tsv'

### optional: create new temp database to work with

In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from sqlmodel import select, delete

db_url = get_db_url()
print(db_url)
engine = get_engine(db_url)
init_db(engine)

# import, using the global 'engine' defined in database module
import_station_file(station_file)

## Create WeatherStation object

In [ ]:
# get the first station from the database for a specific type
def get_one_station(station_type):

    
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()

    return(weather_station)



    


In [ ]:
station_type = 'SPECTRUM'

### create station object from database

In [ ]:
station = get_one_station(station_type)
station.station_type

### OR create weather station object without database

In [ ]:
stations = read_station_table(station_file)
# add code to find the correct one
station_data = list(filter(lambda x: (x['station_type']==station_type), stations))[0]
station = WeatherStation.model_validate(station_data) 
station.station_type


## Test APIs


In [ ]:
# create API class from data file
from ewxpwsdb.weather_apis import API_CLASS_TYPES
wapi = API_CLASS_TYPES[station.station_type](station)
wapi.station_type

In [ ]:
print(wapi.station_type)
print(wapi.sampling_interval)
print(wapi.APIConfigClass)

In [ ]:
# check that configuration class is instantiated with same data in database
api_config = wapi.APIConfigClass.model_validate_json_str(wapi.weather_station.api_config)
print(api_config == wapi.api_config)

In [ ]:
r = wapi.get_readings()


In [ ]:
r[0].response_text

In [ ]:

readings = wapi.transform(r)
readings


In [ ]:
isinstance(readings, list)

## Clean up 

If using databases, remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(StationType))
        session.exec(delete(WeatherStation))
        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()